In [2]:
import json
import open3d as o3d
import numpy as np
import pandas as pd
from pdb import set_trace as bp
from PIL import Image
import random
import os
from tqdm import tqdm

In [13]:
def read_obj(obj_path):
    obj_dict = {}
    count = 0
    with open(obj_path) as file:
        
        line = file.readline()
        strs = line.split(" ")
        while strs[0]!='o':
            line = file.readline()
            strs = line.split(" ")
        while 1:
            if not line:
                break
            strs = line.split(" ")
            if strs[0] =='o':

                name = strs[1]
                obj_dict[name] = {}
                points = []
                faces = []
                normals = []
                
                line = file.readline()
                strs = line.split(" ")
                while strs[0]!='o':

                    if not line:
                        break
                    if strs[0] =='v':
                        points.append((float(strs[1]),float(strs[2]),float(strs[3])))

                    if strs[0] =='vt':
                        pass


                        
                    if strs[0] =='vn':
                        normals.append((float(strs[1]),float(strs[2]),float(strs[3])))
                    if strs[0] =='f':
                        info = strs[1:]

                        if len(info)==3:
                            faces.append((int(info[0].split('/')[0]),int(info[1].split('/')[0]),int(info[2].split('/')[0])))
                        elif len(info)==4:
                            faces.append((int(info[0].split('/')[0]),int(info[1].split('/')[0]),int(info[2].split('/')[0])))
                            faces.append((int(info[0].split('/')[0]),int(info[1].split('/')[0]),int(info[3].split('/')[0])))
                            faces.append((int(info[0].split('/')[0]),int(info[3].split('/')[0]),int(info[2].split('/')[0])))
                            faces.append((int(info[3].split('/')[0]),int(info[1].split('/')[0]),int(info[2].split('/')[0])))
                    line = file.readline()
                    strs = line.split(" ")
                

                
                obj_dict[name]['point'] = np.array(points)
                obj_dict[name]['face'] = np.array(faces)
                obj_dict[name]['normal'] = np.array(normals)
                

    
    return obj_dict

In [4]:
def drawSphere(center, radius, color=[0.0,0.0,0.0]):
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
    transform_mat = np.eye(4)
    transform_mat[0:3, -1] = center
    mesh_sphere.transform(transform_mat)
    mesh_sphere.paint_uniform_color(color)
    return mesh_sphere


def drawCone(bottom_center, top_position, color=[0.6, 0.6, 0.9]):
    cone = o3d.geometry.TriangleMesh.create_cone(radius=0.007, height=np.linalg.norm(top_position - bottom_center)+1e-6)
    line1 = np.array([0.0, 0.0, 1.0])
    line2 = (top_position - bottom_center) / (np.linalg.norm(top_position - bottom_center)+1e-6)
    v = np.cross(line1, line2)
    c = np.dot(line1, line2) + 1e-8
    k = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    R = np.eye(3) + k + np.matmul(k, k) * (1 / (1 + c))
    if np.abs(c + 1.0) < 1e-4: # the above formula doesn't apply when cos(∠(𝑎,𝑏))=−1
        R = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]])
    T = bottom_center + 5e-3 * line2
    #print(R)
    cone.transform(np.concatenate((np.concatenate((R, T[:, np.newaxis]), axis=1), np.array([[0.0, 0.0, 0.0, 1.0]])), axis=0))
    cone.paint_uniform_color(color)
    return cone

In [5]:
def change(pos_array):
    p = pos_array
    return (p[0],p[2],-p[1]+0.2)

In [10]:
def show_pcd(animal_name,method='pcd'):

    obj_dict = read_obj(os.path.join(mesh_dir,'{}/frame_000001.obj'.format(animal_name)))
    with open(os.path.join(weight_dir,'{}_weight_dic.json'.format(animal_name)),'r') as f:
        weight_dict = json.load(f)

    group_name = []
    for key,weight_value in weight_dict.items():

        group_name += weight_value['group_name']

    group_name = list(set(group_name))

    index = [i for i in range(len(group_name))]
    group_name_dict = dict(zip(group_name,index))




    color = [0,0.2,0.4,0.6,0.8,1]
    color_list = []
    for i in range(len(group_name)):
        color_list.append(random.sample(color,3))
    color_array = np.array(color_list)


    info = np.load(os.path.join(skeleton_dir,'{}.npy'.format(animal_name)),allow_pickle=True).item()

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    ctr = vis.get_view_control()

    this_level = [info['def_c_root_joint']]

    while this_level:
        next_level = []
        for p_node in this_level:

            if p_node['parent']==None:
                if p_node['is_leaf']==0:
                    next_level+=[info[info_index] for info_index in  p_node['children']]
                else:
                    continue
                continue
            sp = drawSphere(change(p_node['head']), 0.01, color=[1.0, 0.0, 0.0])
            vis.add_geometry(sp) # [0.3, 0.1, 0.1]
            vis.update_geometry(sp)
            sp = drawSphere(change(p_node['tail']), 0.01, color=[1.0, 0.0, 0.0])
            vis.add_geometry(sp)
            vis.update_geometry(sp)
            con = drawCone(np.array(change(p_node['head'])), np.array(change(p_node['tail'])))
            vis.add_geometry(con)
            vis.update_geometry(con)

            if p_node['is_leaf']==0:
                next_level+=[info[info_index] for info_index in  p_node['children']]
            else:
                continue
        this_level = next_level
            
            
    
    for obj_name,obj in obj_dict.items():
        
        for weight_key in weight_dict.keys():
            if weight_key in obj_name:
                weight = weight_dict[weight_key]
                break
        
        
        
        point_number = len(weight)-1
        
        weight_array = np.zeros((point_number,len(group_name)))
        
        for i in range(len(weight_array)):
            weight_info = weight[str(i)]
            for key,value in weight_info.items():
                weight_array[i,group_name_dict[key]] = value
            
        points = obj['point']
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(weight_array@color_array)
        vis.add_geometry(pcd)
        vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

#     param = o3d.io.read_pinhole_camera_parameters('sideview.json')
#     ctr.convert_from_pinhole_camera_parameters(param)
    vis.run()
    image = vis.capture_screen_float_buffer()

    # param = ctr.convert_to_pinhole_camera_parameters()
    # o3d.io.write_pinhole_camera_parameters('sideview.json', param)

    vis.destroy_window()

    image = np.asarray(image) * 255
    image = image.astype(np.uint8)

    return image

In [16]:
mesh_dir = '/media/ivenwu/My_Disk/video_obj_sample/'
skeleton_dir = '/home/ivenwu/桌面/Planetzoo_files/skeleton_file'
weight_dir = '/home/ivenwu/桌面/Planetzoo_files/weight'
image_save_dir = './image'


In [17]:
if not os.path.exists(image_save_dir):
    os.makedirs(image_save_dir)

In [12]:
for single_file in tqdm(os.listdir(mesh_dir)):
#     animal_name = single_file[:-4]
    animal_name = single_file
    image = show_pcd(animal_name)

    image = Image.fromarray(image)
    image.save(os.path.join(image_save_dir,'{}.png'.format(animal_name)))
    break

  0%|          | 0/2 [00:02<?, ?it/s]
